## Loading dataset 

In [ ]:
import os 
os.environ['KAGGLE_CONFIG_DIR'] = '/content'

In [ ]:
! kaggle datasets download -d rahilmehtaucoe2784/fashion

 97% 185M/191M [00:06<00:00, 24.5MB/s]
100% 191M/191M [00:06<00:00, 33.1MB/s]


In [ ]:
%%capture
!unzip \*.zip && rm *.zip &> /dev/null

## installing and importing Libraries 

In [ ]:
!pip install -q tflite-model-maker
!pip install -q pycocotools

     |████████████████████████████████| 616 kB 4.3 MB/s 
     |████████████████████████████████| 120 kB 91.6 MB/s 
     |████████████████████████████████| 596 kB 81.7 MB/s 
     |████████████████████████████████| 6.4 MB 74.6 MB/s 
     |████████████████████████████████| 1.1 MB 68.7 MB/s 
     |████████████████████████████████| 87 kB 9.2 MB/s 
     |████████████████████████████████| 1.1 MB 55.7 MB/s 
     |████████████████████████████████| 1.2 MB 66.5 MB/s 
     |████████████████████████████████| 234 kB 77.7 MB/s 
     |████████████████████████████████| 77 kB 7.7 MB/s 
     |████████████████████████████████| 3.4 MB 65.9 MB/s 
     |████████████████████████████████| 840 kB 75.1 MB/s 
     |████████████████████████████████| 25.3 MB 1.2 MB/s 
     |████████████████████████████████| 99 kB 12.0 MB/s 
     |████████████████████████████████| 352 kB 79.5 MB/s 
     |████████████████████████████████| 47.7 MB 166 kB/s 
     |████████████████████████████████| 462 kB 95.8 MB/s 
     |██████████████

In [ ]:
import numpy as np
import os

#from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)
from tensorflow import keras

## Training 

In [ ]:
train_image_dir = '/content/Final Img copy 2/train'
train_annotation_dir = '/content/Final Img copy 2/train anotation'
validate_image_dir = '/content/Final Img copy 2/valid'
validate_anaotation_dir = '/content/Final Img copy 2/valid anotation '

In [ ]:

spec = model_spec.get('efficientdet_lite4')

In [ ]:
train_dir = object_detector.DataLoader.from_pascal_voc(train_image_dir, train_annotation_dir, label_map={1:"Design Kurta", 2: "Plain kurta", 3: "Casual Shirt", 4:"Formal Shirt", 5: "Desgin sherwani", 6:"Plain sherwani", 7:"tshirt"})

In [ ]:
validate_dir = object_detector.DataLoader.from_pascal_voc(validate_image_dir, validate_anaotation_dir, label_map={1:"Design Kurta", 2: "Plain kurta", 3: "Casual Shirt", 4:"Formal Shirt", 5: "Desgin sherwani", 6:"Plain sherwani",7:"tshirt"})

In [10]:
model = object_detector.create(train_dir, 
                               model_spec=spec, 
                               epochs=10, 
                               batch_size=5, 
                               train_whole_model=True, 
                               validation_data=validate_dir
                               )

Epoch 1/50
176/176 [==============================] - 240s 950ms/step - det_loss: 1.1308 - cls_loss: 0.9037 - box_loss: 0.0045 - reg_l2_loss: 0.1087 - loss: 1.2395 - learning_rate: 0.0071 - gradient_norm: 2.8845 - val_det_loss: 0.6490 - val_cls_loss: 0.5756 - val_box_loss: 0.0015 - val_reg_l2_loss: 0.1088 - val_loss: 0.7578
Epoch 2/50
176/176 [==============================] - 162s 923ms/step - det_loss: 0.5564 - cls_loss: 0.4943 - box_loss: 0.0012 - reg_l2_loss: 0.1089 - loss: 0.6653 - learning_rate: 0.0062 - gradient_norm: 3.5938 - val_det_loss: 0.5896 - val_cls_loss: 0.5499 - val_box_loss: 7.9544e-04 - val_reg_l2_loss: 0.1090 - val_loss: 0.6986
Epoch 3/50
176/176 [==============================] - 164s 931ms/step - det_loss: 0.4521 - cls_loss: 0.4120 - box_loss: 8.0138e-04 - reg_l2_loss: 0.1090 - loss: 0.5611 - learning_rate: 0.0062 - gradient_norm: 3.3047 - val_det_loss: 0.4261 - val_cls_loss: 0.4049 - val_box_loss: 4.2554e-04 - val_reg_l2_loss: 0.1091 - val_loss: 0.5352
Epoch 4/50

## Evaluating the model 

In [11]:
model.evaluate(validate_dir,batch_size=5)

49/49 [==============================] - 20s 296ms/step



{'AP': 0.8975385,
 'AP50': 0.9326052,
 'AP75': 0.9238078,
 'AP_/Casual Shirt': 0.89329845,
 'AP_/Desgin sherwani': 0.9084253,
 'AP_/Design Kurta': 0.88989013,
 'AP_/Formal Shirt': 0.9520745,
 'AP_/Plain kurta': 0.8550681,
 'AP_/Plain sherwani': 0.8779376,
 'AP_/tshirt': 0.9060753,
 'APl': 0.89775676,
 'APm': -1.0,
 'APs': -1.0,
 'ARl': 0.9587251,
 'ARm': -1.0,
 'ARmax1': 0.9482731,
 'ARmax10': 0.9587251,
 'ARmax100': 0.9587251,
 'ARs': -1.0}

## Saving Model 

In [12]:
model.export(export_dir='/content/')